In [1]:
import pandas as pd
import json

# df = pd.read_csv("data\part-00000-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv")
df = pd.read_csv("data/part-00157-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv")
df.rename(columns={'Magnitue': 'Magnitude'}, inplace=True)
print(df.columns)
print("\n")
print(df.head(5))
print(df["label"].unique())

Index(['flow_duration', 'Header_Length', 'Protocol Type', 'Duration', 'Rate',
       'Srate', 'Drate', 'fin_flag_number', 'syn_flag_number',
       'rst_flag_number', 'psh_flag_number', 'ack_flag_number',
       'ece_flag_number', 'cwr_flag_number', 'ack_count', 'syn_count',
       'fin_count', 'urg_count', 'rst_count', 'HTTP', 'HTTPS', 'DNS', 'Telnet',
       'SMTP', 'SSH', 'IRC', 'TCP', 'UDP', 'DHCP', 'ARP', 'ICMP', 'IPv', 'LLC',
       'Tot sum', 'Min', 'Max', 'AVG', 'Std', 'Tot size', 'IAT', 'Number',
       'Magnitude', 'Radius', 'Covariance', 'Variance', 'Weight', 'label'],
      dtype='object')


   flow_duration  Header_Length  Protocol Type  Duration        Rate  \
0       0.621457          71.28           6.00     64.00    1.029826   
1       0.000000          54.00           6.00     64.00   23.345137   
2       0.000000          54.00           6.00     64.00  195.657228   
3       1.554891         148.50           6.00     64.00    3.122125   
4       0.000000           2.

In [2]:
# Select only 'label' and 'Magnitude' columns
selected_data = df[['label', 'Magnitude']]

# Group by 'label' and aggregate 'Magnitude' values into a list
grouped_data = selected_data.groupby('label')['Magnitude'].agg(list).reset_index()

# Select 1 in 4 entries for every label
selected_data_1_in_4 = grouped_data.groupby('label').apply(lambda x: x.iloc[::4]).reset_index(drop=True)

# Save the selected data to a JSON file
selected_data_1_in_4.to_json('boxplot_data.json', orient='records', lines=False)

In [3]:
# Group the DataFrame by 'label' and convert each group to a list
grouped_data = df.groupby('label')['Duration'].apply(list)

# Convert the grouped data to a list of dictionaries
data = [{'label': label, 'Duration': durations} for label, durations in grouped_data.items()]

# Write the Python object to a file
with open('violin-plot_Duration_data.json', 'w') as f:
    json.dump(data, f, indent=4)

In [4]:
selected_columns = ['label', 'Tot size']
selected_data = df[selected_columns]
# Save the selected data to a JSON file
selected_data.to_json('violin-plot_Size_data.json', orient='records', lines=True)

In [5]:
selected_columns = ['Tot size', 'ack_count']
selected_data = df[selected_columns]
# Save the selected data to a JSON file
selected_data.to_json('scatter-plot_data.json', orient='records', lines=True)

In [6]:
selected_columns = ['Duration', 'Tot size']
selected_data = df[selected_columns]
# Save the selected data to a JSON file
selected_data.to_json('line-plot_data.json', orient='records', lines=True)

In [7]:
selected_columns = ['Duration', 'Tot size', 'label']
grouped_data = df[selected_columns]

# Group by 'label' and aggregate 'Duration' and 'Tot size' values into a list
selected_data = grouped_data.groupby('label').agg(list)

# Select 1 in 4 entries for every label
selected_data = selected_data.apply(lambda x: x.iloc[::3]).reset_index()

# Convert DataFrame to JSON string
json_str = selected_data.to_json(orient='records')

# Parse JSON string to Python object
data = json.loads(json_str)

# Write pretty-printed JSON to file
with open('scatter-plot_Size_data.json', 'w') as f:
    json.dump(data, f, indent=4)

In [8]:
import pandas as pd
import numpy as np

# Step 1: Select numerical columns
numerical_cols = df.select_dtypes(include=np.number).columns.tolist()

# Step 2: Calculate variance for each numerical column and select the top 10
top_variance_cols = df[numerical_cols].var().nlargest(10).index.tolist()

# Step 3: Print the top 10 columns with the highest numeric variance
print("Top 10 Columns with Highest Numeric Variance:")
print(", ".join(top_variance_cols))

# Step 4: Print mean, median, max, min, and standard deviation for each column across the top 10 labels
top_labels = df['label'].value_counts().nlargest(10).index.tolist()

result_data = {}

for col in top_variance_cols:
    result_data[col] = {}  # Initialize an empty dictionary for each column
    print(f"\nStatistics for '{col}' across the top 10 labels:")
    for label in top_labels:
        values = df[df['label'] == label][col]
        # print(f"{label}: Mean = {values.mean():.2f}, Median = {values.median():.2f}, Max = {values.max():.2f}, Min = {values.min():.2f}, Std Dev = {values.std():.2f}")
        print(f"{label}: Mean = {values.mean():.2f}")
        result_data[col][label] = values.mean()

result_json = pd.DataFrame(result_data)
result_json.to_json("mean_data.json", orient="index", indent=4)


Top 10 Columns with Highest Numeric Variance:
IAT, Header_Length, Covariance, Rate, Srate, Tot sum, Max, rst_count, flow_duration, Tot size

Statistics for 'IAT' across the top 10 labels:
DDoS-ICMP_Flood: Mean = 83152343.01
DDoS-UDP_Flood: Mean = 83112939.89
DDoS-TCP_Flood: Mean = 83052210.41
DDoS-PSHACK_Flood: Mean = 83319668.59
DDoS-RSTFINFlood: Mean = 83348220.32
DDoS-SYN_Flood: Mean = 83097347.02
DDoS-SynonymousIP_Flood: Mean = 83360158.18
DoS-UDP_Flood: Mean = 83005684.48
DoS-TCP_Flood: Mean = 82919880.57
DoS-SYN_Flood: Mean = 82947407.27

Statistics for 'Header_Length' across the top 10 labels:
DDoS-ICMP_Flood: Mean = 481.55
DDoS-UDP_Flood: Mean = 25199.86
DDoS-TCP_Flood: Mean = 163.06
DDoS-PSHACK_Flood: Mean = 151.13
DDoS-RSTFINFlood: Mean = 702.42
DDoS-SYN_Flood: Mean = 192.08
DDoS-SynonymousIP_Flood: Mean = 276.46
DoS-UDP_Flood: Mean = 19607.29
DoS-TCP_Flood: Mean = 2655.89
DoS-SYN_Flood: Mean = 2793.02

Statistics for 'Covariance' across the top 10 labels:
DDoS-ICMP_Flood: Me

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Step 1: Select numerical columns
numerical_cols = df.select_dtypes(include=np.number).columns.tolist()

# Remove "IAT" and "Tot sum" from the list of numerical columns
numerical_cols = [col for col in numerical_cols if col not in ["IAT", "Tot sum"]]

# Step 2: Calculate variance for each numerical column (excluding "IAT") and select the top 10
top_variance_cols = df[numerical_cols].var().nlargest(10).index.tolist()

# Step 3: Print the top 10 columns with the highest numeric variance
print("Top 10 Columns with Highest Numeric Variance (excluding 'IAT'):")
print(", ".join(top_variance_cols))

# Step 4: Print mean, median, max, min, and standard deviation for each column across the top 10 labels
top_labels = df['label'].value_counts().nlargest(10).index.tolist()

normalized_data = pd.DataFrame()

# Calculate the dynamic rescale factor for each column
# rescale_factors = {col: 10 ** (int(np.log10(df[col].max())) - 1) for col in top_variance_cols}
rescale_factors = {col: 10 ** (int(np.log10(df[col].max()))) for col in top_variance_cols}

for col in top_variance_cols:
    # Normalize the data between 0 and 1
    normalized_data[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())
    # Rescale the normalized data based on the dynamic factor
    normalized_data[col] *= rescale_factors[col]

    # Filter out values that are significantly larger than the mean of the column
    threshold = 10 * normalized_data[col].median()
    normalized_data[col] = np.where(normalized_data[col] > threshold, threshold, normalized_data[col])

# Print normalized data for verification
print("\nRescaled Data:")
print(normalized_data.head())

# Save the normalized data to a JSON file
normalized_data.to_json("spider-plot_normalized_data.json", orient="index", indent=4)

Top 10 Columns with Highest Numeric Variance (excluding 'IAT'):
Header_Length, Covariance, Rate, Srate, Max, rst_count, flow_duration, Tot size, AVG, Radius

Rescaled Data:
   Header_Length  Covariance       Rate      Srate       Max  rst_count  \
0       7.378044         0.0   0.175378   0.175378  3.600576        0.0   
1       5.589427         0.0   3.975653   3.975653  3.600576        0.0   
2       5.589427         0.0  26.670390  26.670390  3.600576        0.0   
3      15.370925         0.0   0.531695   0.531695  3.600576        0.0   
4       0.221507         0.0   6.076157   6.076157  9.805569        0.0   

   flow_duration  Tot size       AVG  Radius  
0            0.0  2.425271  2.398946     0.0  
1            0.0  2.425271  2.398946     0.0  
2            0.0  2.425271  2.398946     0.0  
3            0.0  2.425271  2.398946     0.0  
4            0.0  0.347622  0.893230     0.0  
